<a href="https://colab.research.google.com/github/imammarzuki/Eksperimen/blob/main/CBF_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix
import numpy as np

# Membuat data pengguna dan item
data = {
    'user_id': [1, 1, 2, 2, 3, 3, 4, 4],
    'item_id': [101, 102, 101, 103, 102, 104, 101, 104],
    'description': [
        'nature beach sunset',
        'historic culture museum',
        'nature mountains hiking',
        'urban shopping nightlife',
        'historic culture museum',
        'nature beach sunset',
        'urban shopping nightlife',
        'nature mountains hiking'
    ],
    'rating': [5, 3, 4, 5, 2, 5, 4, 3]
}

df = pd.DataFrame(data)



In [2]:
df

,user_id,item_id,description,rating
0,1,101,nature beach sunset,5
1,1,102,historic culture museum,3
2,2,101,nature mountains hiking,4
3,2,103,urban shopping nightlife,5
4,3,102,historic culture museum,2
5,3,104,nature beach sunset,5
6,4,101,urban shopping nightlife,4
7,4,104,nature mountains hiking,3


In [3]:
# Menghitung TF-IDF dari deskripsi
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['description'])

In [4]:
# Menghitung cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Fungsi untuk mendapatkan rekomendasi
def get_recommendations(user_id, cosine_sim=cosine_sim):
    # Mendapatkan indeks item yang telah dinilai oleh pengguna
    idx = df.index[df['user_id'] == user_id].tolist()

    # Mengumpulkan skor kesamaan untuk item tersebut
    sim_scores = list(enumerate(cosine_sim[idx].mean(axis=0)))

    # Mengurutkan item berdasarkan skor kesamaan
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Mendapatkan skor item yang paling tinggi yang belum dinilai
    sim_scores = [sim for sim in sim_scores if sim[0] not in idx]

    # Mendapatkan item_id
    item_indices = [i[0] for i in sim_scores]

    return df['item_id'].iloc[item_indices].tolist()

# Mendapatkan rekomendasi untuk pengguna 1
recommendations = get_recommendations(1)
print("Recommendations:", recommendations)


Recommendations: [102, 104, 101, 104, 103, 101]


In [5]:
# Misal, skor aktual dan prediksi untuk pengguna 1
actual = df[df['user_id'] == 1]['rating'].tolist()
predicted = [5, 3]  # Ini harusnya dari model, di sini hanya sebagai contoh

# Menghitung MAE dan RMSE
mae = mean_absolute_error(actual, predicted)
rmse = np.sqrt(mean_squared_error(actual, predicted))
print("MAE:", mae)
print("RMSE:", rmse)

# Confusion Matrix (Biasanya untuk klasifikasi, tetapi kita contohkan dengan rating yang dibulatkan)
cm = confusion_matrix(actual, predicted, labels=[1, 2, 3, 4, 5])
print("Confusion Matrix:\n", cm)


MAE: 0.0
RMSE: 0.0
Confusion Matrix:
 [[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 1]]
